In [11]:
## 将所有基因的表达整理为log2(TPM+1)的形式
tissues = ['Anther', 'Cotyledon', 'Ovule',  'Hypocotyl', 'Leaf', 'Petal', 'Radicle', 'Root', 'Stem', 'Stigma',  'Fiber_10DPA', 'Fiber_20DPA']
import pandas as pd
import numpy as np
df_result = pd.DataFrame()
for t in tissues:
    if 'Fiber' in t or "Radicle" in t:
        n = 3
    else:
        n = 4
    for i in range(1, n):
        input_f = "G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/expression_gene/all_FPKM_TPM/{0}_rep{1}_FPKM_TPM.txt".format(t, i)
        cols = ['GeneId', 'Chromosome', 'Start', 'End', 'FPKM', '{0}{1}'.format(t, i)]
        cols_C = ['GeneId', 'Chromosome', 'Start', 'End', '{0}{1}'.format(t, i)]
        df_tmp = pd.read_csv(input_f, sep="\t", names=cols)
        df_tmp = df_tmp[cols_C].copy()
        df_tmp.drop_duplicates(keep='first', subset=['GeneId'], inplace=True)
        df_tmp['{0}{1}'.format(t, i)] = df_tmp['{0}{1}'.format(t, i)].apply(lambda x: np.log2(x+1))
        if df_result.shape[0] == 0:
            df_result = df_tmp.copy()
        else:
            df_result = pd.merge(df_result, df_tmp, how='inner', on=['GeneId', 'Chromosome', 'Start', 'End'])            
#df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/expression_gene/all_FPKM_TPM/J668_all_tissues_TPM.bed", sep="\t", index=False)
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/J668_all_tissues_log2TPM1.bed", sep="\t", index=False)

In [ ]:
## 分析WGCNA 网络中所能够包含的TAU 基因
import pandas as pd
import sys 
Netf = sys.argv[1]
Nodef = sys.argv[2]
TAUf = sys.argv[3]
out_netf = sys.argv[4]
out_nodef = sys.argv[5]

df_net = pd.read_csv(Netf, sep="\t", names=['source', 'target', 'weight'])
df_node = pd.read_csv(Nodef, sep="\t", names=['ID', 'module'])
df_TAU = pd.read_csv(TAUf, sep="\t")

def choose(df1, df2, tags):
    if tags == 'node':
        df2.columns =['ID', 'TSG']
        df_return = pd.merge(df1, df2, how='inner', on=['ID'])
    if tags == 'net':
        df2 = df2[['GeneId']]
        df2.columns = ['source']
        df_one = pd.merge(df1, df2, how='inner', on=['source'])
        df2.columns = ['target']
        df_return = pd.merge(df_one, df2, how='inner', on=['target'])
    return df_return
df_TAU_net = choose(df_net, df_TAU, 'net')
df_TAU_node = choose(df_node, df_TAU, 'node')
df_TAU_net['type'] = ['Undirected']*df_TAU_net.shape[0]
df_TAU_net.to_csv(out_netf, index=False)
df_TAU_node.to_csv(out_nodef, index=False)
 

In [5]:
import pandas as pd
df = pd.read_csv('G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/module/Go_term/Goterm_node2.csv')
print(df.shape[0])
df_result = df.drop_duplicates(keep='first', subset=['node.name'])
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/module/Go_term/Goterm_node3.csv", index=False)

90


In [ ]:
## 分析TAU 基因在WGCNA 各module 中的占比
import pandas as pd
df_WGCNA_module = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/module/J668_all_tissue.module.txt", sep="\t", names=['GeneId', 'Module'])
df_TAU = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/tau_gini/J668_tissue_special_gene.txt", sep="\t")
df_TAU = df_TAU[['GeneId', 'tags']].copy()

## 创建一个比例统计函数
def sta_ratio(df, tissue):
    df['number'] = [1]*df.shape[0]
    total = df.shape[0]
    df_return = df.groupby(by=['Module'], as_index=False)['number'].sum()
    df_return['ratio'] = 100*(df_return['number']/total)
    df_return['Tissue'] = [tissue]*df_return.shape[0]
    return df_return
## 分析组织特异性表达基因在于各module 中的分布
tissues = ['anther', 'leaf', 'hypocotyl', 'cotyledon', 'stem', 'ovule', 'root', 'radicle', 'fiber_10DPA', 'fiber_20DPA', 'petal', 'stigma']
df_result = pd.DataFrame()
for t in tissues:
    df_TAU_GeneId = df_TAU.query("tags==@t")[['GeneId']]
    df_TAU_WGCNA = pd.merge(df_WGCNA_module, df_TAU_GeneId, how='inner', on=['GeneId'])
    df_TAU_module = sta_ratio(df_TAU_WGCNA, t)
    df_result = pd.concat([df_result, df_TAU_module], axis=0)
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/module/TAU_ratio/TAU_WGCNA_module_ratio.bed", sep="\t", index=False)

In [13]:
## 分析WGCNA各module中TAU基因占比
import pandas as pd
df_WGCNA_module = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/module/J668_all_tissue.module.txt", sep="\t", names=['GeneId', 'Module'])
df_TAU = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/tau_gini/J668_tissue_special_gene.txt", sep="\t")
df_TAU = df_TAU[['GeneId', 'tags']].copy()
df_TAU = df_TAU.query("tags!='fiber_5DPA'").copy()
old_module_tags = ['red',  'yellow', 'green', 'brown', 'lightyellow', 'turquoise', 'pink',  'midnightblue', 'lightcyan','greenyellow', 'magenta', 'salmon',  
                   'darkgreen', 'black', 'blue', 'cyan', 'darkgrey','darkred', 'darkturquoise', 'grey', 
                   'grey60', 'lightgreen',  'orange',   'purple', 'royalblue', 'tan']
new_module_tags = ['Module1', 'Module2', 'Module3', 'Module4', 'Module5', 'Module6', 'Module7', 'Module8', 'Module9', 'Module10', 'Module11', 'Module12',
                   'Module13', 'Module14', 'Module15', 'Module16', 'Module17', 'Module18', 'Module19', 'Module20', 'Module21', 'Module22', 'Module23', 
                   'Module24', 'Module25', 'Module26']
df_WGCNA_TAU = pd.merge(df_WGCNA_module, df_TAU, how='left', on=['GeneId'])
df_WGCNA_TAU.fillna('Not TSG', inplace=True)
#df_WGCNA_TAU.replace(old_module_tags, new_module_tags, inplace=True)
df_WGCNA_TAU['number'] = [1]*df_WGCNA_TAU.shape[0]
df_result = df_WGCNA_TAU.groupby(by=['Module', 'tags'], as_index=False)['number'].sum()
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/module/TAU_ratio/WGCNA_TAU_module_ratio.bed", sep="\t", index=False)

In [ ]:
#分析同源bias 基因所相关的共表达基因平均表达水平
import pandas as pd
import numpy as np
df_hg_bias = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/homo_gene2/gene_bias/all_tissue_bias_gene_pair.bed", sep="\t")
df_WGCNA = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/J668_all_tissue_005.edges.txt", sep="\t")
df_TPM = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/expression_gene/J668_all_tissue_average_TPM_long.txt", sep="\t")
df_anther = df_hg_bias.query("tags=='Anther'").copy()


In [36]:

# 分析一个基因其所包含的所有共表达基因的平均值
import re
def gene_WGCNA_average(df_WGCNA, df_TPM, geneid):
    # chrom = re.findall(r'Ghir_\w{3}')[0]
    # df_WGCNA = df_WGCNA.query("fromNode.str.contain(@chrom) | toNode.str.contain(@chrom)", engine='python')
    df_WGCNA_gene = df_WGCNA.query("fromNode==@geneid|toNode==@geneid")[['fromNode', 'toNode']].copy()
    #print(df_WGCNA_gene.shape[0])
    if df_WGCNA_gene.shape[0]==0:
        gene_exp = df_TPM.query("GeneId==@geneid")['TPM'].mean()
        #print(gene_exp)
        return gene_exp
    else:
        df_WGCNA_gene_np = df_WGCNA_gene.to_numpy()
        df_np = df_WGCNA_gene_np.reshape(-1,1)
        df_WGCNA_gene = pd.DataFrame(df_np, columns = ['GeneId'])
        df_WGCNA_gene.drop_duplicates(keep=False, inplace=True)
        df_WGCNA_gene_TPM = pd.merge(df_WGCNA_gene, df_TPM, how='inner', on=['GeneId'])
        #print(df_WGCNA_gene_TPM['TPM'].mean())
        return df_WGCNA_gene_TPM['TPM'].mean()
df_anther_TPM = df_TPM.query("tags=='Anther'").copy()
df_anther_HG = df_hg_bias.query("tags=='Anther'").copy()
df_result = pd.DataFrame(columns=['GeneId', 'TPM', 'tags'])
for index, row in df_anther_HG.iterrows():
    geneid_at,  geneid_dt, sig = row['GeneId_At'], row['GeneId_Dt'], row['sig']
    geneid_at_tpm = gene_WGCNA_average(df_WGCNA, df_anther_TPM, geneid_at)
    geneid_dt_tpm = gene_WGCNA_average(df_WGCNA, df_anther_TPM, geneid_dt)
    df_result[df_result.shape[0]] = [geneid_at, geneid_at_tpm, sig]
    df_result[df_result.shape[0]] = [geneid_dt, geneid_dt_tpm, sig]
    #print([geneid_at, geneid_at_tpm, sig])
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/HG_bias/Anther_HG_WGCNA_average_TPM", sep="\t")
# gene_WGCNA_average(df_WGCNA, df_anther_TPM, 'Ghir_A01G000330')
# gene_WGCNA_average(df_WGCNA, df_anther_TPM, 'Ghir_D01G000340')

In [72]:
## 确定每个WGCNA 基因所连接基因的TPM 平均值
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/J668_all_tissue_005.edges.txt", sep="\t")
df_TPM = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/expression_gene/J668_all_tissue_average_TPM_long.txt", sep="\t")
df_TPM = df_TPM.query("tags=='Anther'").copy()
df_TPM = df_TPM[['GeneId', 'TPM']].copy()
df_TPM.columns = ['toNode', 'TPM']
df_a = df[['toNode', 'fromNode', 'weight']].copy()
df_a.columns = ['fromNode', 'toNode', 'weight']
df_WGCNA_pairs = pd.concat([df, df_a], axis=0)
df_WGCNA_pairs = pd.merge(df_WGCNA_pairs, df_TPM, how='left', on=['toNode'])
dz = df_WGCNA_pairs.query("fromNode=='Ghir_A01G000010' | toNode =='Ghir_A01G000010'")
df_result = df_WGCNA_pairs.groupby(by=['fromNode'], as_index=False)['TPM'].mean()
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/J668_all_tissue_WGCNA_average_TPM.csv", index=False)

In [4]:
## 分析能发生bias表达的基因其 共表达基因TPM的平均值
import pandas as pd
df_bias = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_seq/homo_gene2/gene_bias/all_tissue_bias_gene_pair.bed", sep="\t")
df_WGCNA = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/J668_all_tissue_WGCNA_average_TPM.csv")
df_Anther_bias = df_bias.query("tissues=='Anther'").copy()
df_result = pd.DataFrame(columns = ['GeneId', 'TPM', 'Bias', 'Subg'])
for index, row in df_Anther_bias.iterrows():
    gene_At, gene_Dt, at_tpm, dt_tpm , tags = row['GeneId_At'], row['GeneId_Dt'], row['TPM_At'], row['TPM_Dt'], row['sig']
    at_wgcna, dt_wgcna = df_WGCNA.query('fromNode==@gene_At'), df_WGCNA.query('fromNode==@gene_Dt')
    #print(at_wgcna)
    if at_wgcna.shape[0]==0:
        wgcna_tmp_at = at_tpm
    else:
        wgcna_tmp_at = at_wgcna['TPM'].mean()
    if dt_wgcna.shape[0]==0:
        wgcna_tmp_dt = dt_tpm
    else:
        wgcna_tmp_dt = dt_wgcna['TPM'].mean()    
    
    df_result.loc[df_result.shape[0]] = [gene_At, wgcna_tmp_at, tags, 'At']
    df_result.loc[df_result.shape[0]] = [gene_Dt, wgcna_tmp_dt, tags, 'Dt']
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/HG_bias/J668_anther_WGNCA_bias.csv", index=False)

In [91]:
df_a = df_Anther_bias[['GeneId_At']].copy()
df_b = df_Anther_bias[['GeneId_Dt']].copy()
df_a.columns = ['GeneId']
df_b.columns = ['GeneId']
df_c = pd.concat([df_a, df_b], axis=0)
print(df_c.shape[0])
df_WGCNA.columns = ['GeneId', 'TPM']
dfk = pd.merge(df_WGCNA, df_c, how='inner', on=['GeneId'])
print(dfk.shape[0])

12612
9292
